<a href="https://colab.research.google.com/github/Davidfdaf/Optimization-modeling-course/blob/main/Evan_bond_opt_rewrite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PuLP

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime
import math
import numpy as np
from pulp import *

In [ ]:
#params
Portfolio=None
Arated=.20
duration=5.25
Subs=3
bonds=25
oneyearmature=0
LadderStart=3
LadderLength=7
MaxPerYear=5
maxladderlength=11

sectorConstraints = {
    'CS':.2,
    'CD':.2,
    'CST':.2,
    'F':9/25,
    'HC':.2,
    'I':.2,
    'IT':.2,
    'U':.15
}


#Read in data, previous port and ceate lpclass
df = pd.read_excel("IG Universe.xlsx")

now = datetime.now().strftime('%Y-%m-%d %H %M %S')
prob = LpProblem("Corporate_Bond_Portfolio", LpMaximize)              
if Portfolio is not None:
        df = pd.concat([Portfolio, df]).drop_duplicates(subset='CUSIP',keep='last')
        pcusips = list(Portfolio['CUSIP'])
        ebonds=len(pcusips)
        df['In Portfolio'] = df['CUSIP'].isin(pcusips).astype(int)

#preprocess data, set up dummy variables
df = (df .assign(**{"Seniority Level" : lambda x: (x['Seniority Level'] == 'Subordinate').astype(int)}) #have to use this weird dict thing to get the space in the name
         .assign(Rating = lambda x: (x['Rating'].str.contains("A") == True).astype(int))
         .assign(Invested = lambda x: 1))
df = pd.concat([df,pd.get_dummies(df['Industry'], prefix='Sector')],axis=1)


#get bond characteristics into a dict of dicts, nested dict
Cusips = list(df['CUSIP'])
#include Ask Yield To Worst to dict    #yields = dict(zip(Cusips, df['Ask Yield To Worst']))
aspects = dict.fromkeys(['YTW','Rating','Maturity','Duration','Seniority Level','Invested','Ticker'], 0)
for n,i in enumerate(aspects):
    aspects[i] = dict(zip(Cusips, df[i]))
    
#use nested dicts to store bonds by sector
sectors = dict.fromkeys(['CS','CD','CST','E','F','HC','I','IT','M','U'], 0)
sector_names = ['Sector_Communication Services', 'Sector_Consumer Discretionary', 'Sector_Consumer_Staples', 'Sector_Energy', 'Sector_Financials', \
                'Sector_Health Care', 'Sector_Industrials', 'Sector_Information Technology', 'Sector_Materials', 'Sector_Utilities']
for n,i in enumerate(sectors):
    if sector_names[n] in df.columns:
        sectors[i] = dict(zip(Cusips, df[sector_names[n]]))
        

#create new variables and set broad constraints liking rating, subordinate, and duration
Symbols = list(df['Ticker'])
Arated = math.floor(Arated*bonds)
LadderYears = relativedelta(df['Maturity'].max(), df['Maturity'].min()).years
Cusip_var = LpVariable.dicts("Cusips", Cusips, upBound=1.0, lowBound=0, cat='Integer')
prob += lpSum([aspects['YTW'][i] * Cusip_var[i] for i in Cusip_var])
if Portfolio is not None:
    EP = dict(zip(Cusips,df['In Portfolio']))
    prob += lpSum([EP[i] * Cusip_var[i] for i in Cusip_var]) == ebonds
prob += lpSum([aspects['Invested'][i] * Cusip_var[i] for i in Cusip_var]) == bonds
prob += lpSum([aspects['Rating'][i] * Cusip_var[i] for i in Cusip_var]) >= Arated
prob += lpSum([aspects['Duration'][i] * Cusip_var[i] for i in Cusip_var]) <= (duration * bonds)
prob += lpSum([aspects['Seniority Level'][i] * Cusip_var[i] for i in Cusip_var]) <= Subs

#set sector constraints
# upper_bounds = {'CS': math.ceil(.2*bonds),
#                 'CD': math.ceil(.2*bonds),
#                 'CST': math.ceil(.2*bonds),
#                 'E': 1,
#                 'F': math.ceil(9/25*bonds),
#                 'HC': math.ceil(.2*bonds),
#                 'I': math.ceil(.2*bonds),
#                 'IT': math.ceil(.2*bonds),
#                 'M': 1,
#                 'U': math.ceil(.15*bonds)}
upper_bounds = {}
for n,i in enumerate(sectors):
    if i in sectorConstraints:
        upper_bounds[i] = math.ceil(sectorConstraints[i]*bonds)
    else:
        upper_bounds[i] = 1
for n in upper_bounds:
    if sectors[n] != 0:
        prob += lpSum([sectors[n][i] * Cusip_var[i] for i in Cusip_var]) <= upper_bounds[n]

#impliment some maturity and ticker constraints
g2 = datetime.now()
ed = datetime.now().replace(month=12, day=31)
if oneyearmature > 0:
    prob += oneyearmature == lpSum([Cusip_var[i] for i in Cusip_var if g2 <= aspects['Maturity'][i] <= g2 + relativedelta(years=1)])
for s in Symbols:
    prob += 1 >= lpSum([Cusip_var[i] for i in Cusip_var if aspects['Ticker'][i] == s])
prob += 1 >= lpSum([Cusip_var[i] for i in Cusip_var if aspects['Ticker'][i] == 'CVS' or aspects['Ticker'][i] =='WBA'])

#set bond ladder constraints, I don't quite get what they are doing though
if Portfolio is None:
    for a in range(LadderStart, LadderLength):
        g = g2 + relativedelta(years=a)
        g1 = g2 + relativedelta(years=a + 1)
        rb = ed + relativedelta(years=a)
        ry = ed + relativedelta(years=a + 1)
        prob += 2 <= lpSum([Cusip_var[i] for i in Cusip_var if g <= aspects['Maturity'][i] <= g1])
        prob += MaxPerYear >= lpSum([Cusip_var[i] for i in Cusip_var if g <= aspects['Maturity'][i] <= g1])
        prob += 2 <= lpSum([Cusip_var[i] for i in Cusip_var if rb <= aspects['Maturity'][i] <= ry])
        
#check if this is a redundant constraint or if it can be written more simply
for a in range(LadderStart, LadderYears + 1):
    rb = ed + relativedelta(years=a)
    ry = ed + relativedelta(years=a + 1)
    prob += MaxPerYear >= lpSum([Cusip_var[i] for i in Cusip_var if rb <=  aspects['Maturity'][i] <= ry])
    
prob += 0 == lpSum([Cusip_var[i] for i in Cusip_var if  aspects['Maturity'][i] >= g2 + relativedelta(years=maxladderlength)])
prob.solve()
portfolio = []
#had to add additional statement because at the end of prob.variables I got a ___dummy which is of nonetype
for v in prob.variables():
    if (type(v.varValue)==int) | (type(v.varValue)==float):
        if v.varValue > 0:
            portfolio.append(v.name[-9:])

OptimalPortfolio = df[df['CUSIP'].isin(portfolio)][aspects.keys()]

In [ ]:
print(np.average(OptimalPortfolio['YTW'])*100)

In [ ]:
print(OptimalPortfolio)